# Julia 機器學習：GLM 線性迴歸

## 作業 027：波士頓房價預測資料集

請使用 GLM 中的模型，建立一個預測模型來預測波士頓的房價。

In [22]:
using GLM, RDatasets, MLDataUtils

## 讀取資料

In [23]:
boston = dataset("MASS", "Boston")
first(boston, 10)

,Crim,Zn,Indus,Chas,NOx,Rm,Age,Dis,Rad,Tax
,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Int64,Int64
1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.09,1,296
2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242
3,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242
4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222
5,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222
6,0.02985,0.0,2.18,0,0.458,6.43,58.7,6.0622,3,222
7,0.08829,12.5,7.87,0,0.524,6.012,66.6,5.5605,5,311
8,0.14455,12.5,7.87,0,0.524,6.172,96.1,5.9505,5,311
9,0.21124,12.5,7.87,0,0.524,5.631,100.0,6.0821,5,311


In [26]:
boston[!, :LogMedV] = log.(boston[!, :MedV])

506-element Array{Float64,1}:
 3.1780538303479458
 3.0726933146901194
 3.5467396869528134
 3.5085558999826545
 3.5890591188317256
 3.3568971227655755
 3.131136910560194
 3.299533727885655
 2.803360380906535
 2.9391619220655967
 2.70805020110221
 2.9391619220655967
 3.077312260546414
 ⋮
 3.1986731175506815
 3.139832617527748
 2.9806186357439426
 2.9069010598473755
 3.054001181677967
 2.8622008809294686
 2.8213788864092133
 3.109060958860994
 3.0252910757955354
 3.173878458937465
 3.091042453358316
 2.4765384001174837

In [27]:
indecies = MLDataUtils.shuffleobs(collect(1: nrow(boston)))
train_ind, test_ind = MLDataUtils.splitobs(indecies, at = 0.8)

train = boston[train_ind, :]
test = boston[test_ind, :]

,Crim,Zn,Indus,Chas,NOx,Rm,Age,Dis,Rad,Tax
,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Int64,Int64
1,11.9511,0.0,18.1,0,0.659,5.608,100.0,1.2852,24,666
2,2.44953,0.0,19.58,0,0.605,6.402,95.2,2.2625,5,403
3,0.47547,0.0,9.9,0,0.544,6.113,58.8,4.0019,4,304
4,9.18702,0.0,18.1,0,0.7,5.536,100.0,1.5804,24,666
5,0.06417,0.0,5.96,0,0.499,5.933,68.2,3.3603,5,279
6,0.03466,35.0,6.06,0,0.4379,6.031,23.3,6.6407,1,304
7,0.04527,0.0,11.93,0,0.573,6.12,76.7,2.2875,1,273
8,3.83684,0.0,18.1,0,0.77,6.251,91.1,2.2955,24,666
9,6.96215,0.0,18.1,0,0.7,5.713,97.0,1.9265,24,666


In [30]:
model = lm(@formula(LogMedV ~ Crim + Zn + Indus + Chas + NOx + Rm + Age + Dis + Rad + Tax + PTRatio + Black + LStat), train)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Array{Float64,1}},GLM.DensePredChol{Float64,LinearAlgebra.Cholesky{Float64,Array{Float64,2}}}},Array{Float64,2}}

LogMedV ~ 1 + Crim + Zn + Indus + Chas + NOx + Rm + Age + Dis + Rad + Tax + PTRatio + Black + LStat

Coefficients:
────────────────────────────────────────────────────────────────────────────────────
                    Coef.   Std. Error       t  Pr(>|t|)     Lower 95%     Upper 95%
────────────────────────────────────────────────────────────────────────────────────
(Intercept)   4.06302      0.228297      17.80    <1e-51   3.61418       4.51187
Crim         -0.0118683    0.00168075    -7.06    <1e-11  -0.0151727    -0.00856383
Zn            0.00102522   0.000603793    1.70    0.0903  -0.000161866   0.00221231
Indus         0.00207218   0.00277424     0.75    0.4556  -0.00338212    0.00752648
Chas          0.107329     0.0391827      2.74    0.0064   0.0302935     0.184364
NOx          -0.795002     0.177106      -4.4

In [32]:
exp.(predict(model, test))

101-element Array{Float64,1}:
 18.23458613304072
 25.13771134873606
 20.506012162776557
 12.975089030115319
 23.40446453280581
 23.30043523134143
 21.855273057383076
 17.571374773113355
 16.124796839075934
 14.489623593743511
 11.146498886726217
 33.960098407702795
 23.835627757684293
  ⋮
 22.291934184862534
 18.293904190478244
 25.457042314889755
 15.811411956850467
 21.322789528215658
 16.664186820159706
 33.666394388501764
 23.74512057994066
 15.79412230656674
 14.590354793396187
 42.66246677974318
 16.55918719096062

In [35]:
adjr²(model)

0.7821671785892763